## 📦 Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
import torch
torch.cuda.empty_cache()

## 📁 Data Preparation

In [ ]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Only resizing and normalization for validation
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = data_dir = "C:\\Users\\samia\\Documents\\VS code\\CSE366_Research\\dataset_mini\\MRI\\Training"
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform_train)

# Splitting into train and validation
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply val transforms separately
val_dataset.dataset.transform = transform_val

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
C:\Users\samia\AppData\Local\Temp\ipykernel_2356\2227684297.py:19: SyntaxWarning: invalid escape sequence '\s'
  data_dir = "C:\\Users\samia\\OneDrive\\Desktop\\dataset_mini\\MRI\Training"


## 🧠 Model Definition

In [7]:
class HybridModel(nn.Module):
    def __init__(self, num_classes=4):
        super(HybridModel, self).__init__()

        # Load pre-trained VGG16 (remove classifier)
        vgg16 = models.vgg16(pretrained=True)
        self.vgg16_features = vgg16.features
        self.vgg16_avgpool = vgg16.avgpool
        self.vgg16_fc = nn.Sequential(*list(vgg16.classifier.children())[:-3])  # output: 4096 -> 512

        # Load pre-trained MobileNetV2 (remove classifier)
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.mobilenet_features = mobilenet.features
        self.mobilenet_avgpool = nn.AdaptiveAvgPool2d((1, 1))  # flatten to (batch, 1280)

        # Projection layers
        self.vgg16_proj = nn.Linear(4096, 512)
        self.mobilenet_proj = nn.Linear(1280, 1280)

        # Attention mechanism
        self.attention_fc = nn.Sequential(
            nn.Linear(1792, 512),
            nn.ReLU(),
            nn.Linear(512, 1792),
            nn.Sigmoid()
        )

        # Final classifier
        self.classifier = nn.Linear(1792, num_classes)

    def forward(self, x):
        # VGG16 path
        vgg = self.vgg16_features(x)
        vgg = self.vgg16_avgpool(vgg)
        vgg = torch.flatten(vgg, 1)
        vgg = self.vgg16_fc(vgg)
        vgg = self.vgg16_proj(vgg)  # shape: [B, 512]

        # MobileNetV2 path
        mobile = self.mobilenet_features(x)
        mobile = self.mobilenet_avgpool(mobile)
        mobile = torch.flatten(mobile, 1)
        mobile = self.mobilenet_proj(mobile)  # shape: [B, 1280]

        # Combine features
        combined = torch.cat((vgg, mobile), dim=1)  # shape: [B, 1792]

        # Attention
        attention = self.attention_fc(combined)
        attended = combined * attention

        # Classification
        output = self.classifier(attended)
        return output


## ⚙️ Training Setup

In [10]:
model = HybridModel(num_classes=4).to(device)

# Use CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


C:\Users\samia\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\samia\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\samia\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `w

In [12]:
try:
    from torchinfo import summary
    model = HybridModel()  # Instantiate the model
    print(summary(model, input_size=(1, 3, 224, 224)))
except ImportError:
    print("Install 'torchinfo' for model summary.")


Layer (type:depth-idx)                             Output Shape              Param #
HybridModel                                        [1, 4]                    --
├─Sequential: 1-1                                  [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                                 [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                                   [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                                 [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                                   [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                              [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                                 [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                                   [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                                 [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                                   [1, 128, 112, 112]        --
│    └─MaxPool2d: 2

## 🏋️ Training the Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np
import torch

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5, patience=3):
    best_val_loss = float('inf')
    counter = 0
    history = {
        'train_loss': [], 'val_loss': [],
        'train_accuracy': [], 'val_accuracy': [],
        'train_precision': [], 'val_precision': [],
        'train_recall': [], 'val_recall': []
    }

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        # Training
        model.train()
        train_losses = []
        y_true_train, y_pred_train = [], []

        for inputs, labels in tqdm(train_loader, desc='Training', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            preds = outputs.argmax(dim=1)
            y_true_train.extend(labels.cpu().numpy())
            y_pred_train.extend(preds.cpu().numpy())

        # Compute training metrics
        train_loss = np.mean(train_losses)
        train_acc = accuracy_score(y_true_train, y_pred_train)
        train_prec = precision_score(y_true_train, y_pred_train, average='macro', zero_division=0)
        train_rec = recall_score(y_true_train, y_pred_train, average='macro', zero_division=0)

        # Validation
        model.eval()
        val_losses = []
        y_true_val, y_pred_val = [], []

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc='Validating', leave=False):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_losses.append(loss.item())
                preds = outputs.argmax(dim=1)
                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(preds.cpu().numpy())

        # Compute validation metrics
        val_loss = np.mean(val_losses)
        val_acc = accuracy_score(y_true_val, y_pred_val)
        val_prec = precision_score(y_true_val, y_pred_val, average='macro', zero_division=0)
        val_rec = recall_score(y_true_val, y_pred_val, average='macro', zero_division=0)

        # Logging
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
        history['train_precision'].append(train_prec)
        history['val_precision'].append(val_prec)
        history['train_recall'].append(train_rec)
        history['val_recall'].append(val_rec)

        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'best_model.pth')  # Save best model
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    return history


: 

In [ ]:
# Example: train for 5 epochs
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5, patience=3)


Epoch 1/5


Training:   0%|          | 0/8 [00:00<?, ?it/s]

## 📈 Accuracy Plot

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(16, 10))

    # Accuracy
    plt.subplot(2, 2, 1)
    plt.plot(epochs, history['train_accuracy'], label='Train Accuracy')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(2, 2, 2)
    plt.plot(epochs, history['train_loss'], label='Train Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Precision
    plt.subplot(2, 2, 3)
    plt.plot(epochs, history['train_precision'], label='Train Precision')
    plt.plot(epochs, history['val_precision'], label='Validation Precision')
    plt.title('Precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.legend()

    # Recall
    plt.subplot(2, 2, 4)
    plt.plot(epochs, history['train_recall'], label='Train Recall')
    plt.plot(epochs, history['val_recall'], label='Validation Recall')
    plt.title('Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
plot_history(history)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_dataset = datasets.ImageFolder(data_dir = "C:\\Users\\samia\\Documents\\VS code\\CSE366_Research\\dataset_mini\\MRI\\Testing"
, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Print metrics
print(classification_report(y_true, y_pred, target_names=test_dataset.classes))
